# Data preprocess

In [1]:
# Preprocess 1 Raw to database
# ---------------------------------------------
import sqlite3, pandas, dask.dataframe as dd 

## Load data
rawAnswer = pandas.read_csv("data/Answers.csv", encoding="latin1")
rawQuestions = pandas.read_csv("data/Questions.csv",encoding="latin1")
rawTags = pandas.read_csv("data/Tags.csv", encoding="latin1")
print("load data")

## Drop useless columns
rawAnswer.drop(columns = ["OwnerUserId"])
rawQuestions.drop(columns = ["ClosedDate", "ClosedDate"])
print("Drop useless columns")

## Count contents
for i in range(len(rawAnswer)):
    rawAnswer.at[i, "Body"] = len(str(rawAnswer.loc[i]["Body"]).split(" "))

for i in range(len(rawQuestions)):
    rawQuestions.at[i, "Body"] = len(str(rawQuestions.loc[i]["Body"]).split(" "))
    rawQuestions.at[i, "Title"] = len(str(rawQuestions.loc[i]["Title"]).split(" "))
print("Count contents")

## Store data in a DB
cnx = sqlite3.connect('data/QAT.db')

rawAnswer.to_sql(name='Answer', con=cnx,  if_exists = "replace")
rawQuestions.to_sql(name='Question', con=cnx,  if_exists = "replace")
rawTags.to_sql(name='Tag', con=cnx,  if_exists = "replace")
print("Store data in a DB")
# ---------------------------------------------

load data
Drop useless columns
Count contents
Store data in a DB


In [2]:
# Preprocess 2 Remove redundancy
# Some of the Question has no answer, should be removed
# ---------------------------------------------

Answer = dd.read_sql_table('Answer', "sqlite:///data/QAT.db", index_col='index')
Tag = dd.read_sql_table('Tag', "sqlite:///data/QAT.db", index_col='index')
Question = dd.read_sql_table('Question', "sqlite:///data/QAT.db", index_col='index')

## Get Parent Id from Answer
pid = list(Answer["ParentId"])
pid = set(pid)
## Checking the existence
Question = Question[Question.Id.isin(pid)]
Question = Question.compute()
Answer = Answer.compute()
print('remove unanswered questions')

def column_time_text_to_datetime(df,label):
    to_datetime = []
    for idx, row_data in df.iterrows():
        date_time_obj = datetime.datetime.strptime(row_data[label], '%Y-%m-%dT%H:%M:%SZ')
        to_datetime.append(date_time_obj.timestamp())
    df = df.drop(columns=[label])
    df.insert(len(df.columns),label+'_to_datetime',to_datetime, True)
    return df

Answer = column_time_text_to_datetime(Answer,"CreationDate")
Question = column_time_text_to_datetime(Question,"CreationDate")
print("change time text to float timestamp")

## Store to DB
Question.to_sql("Question_trimmed", "sqlite:///data/QAT_trimmed.db", if_exists = "replace")
Answer.to_sql("Answer_trimmed", "sqlite:///data/QAT_trimmed.db", if_exists = "replace")
# ---------------------------------------------

remove unanswered questions
change time text to float timestamp
